# Day 8: Seven Segment Search

In [1]:
from pathlib import Path
from collections import Counter
from typing import Callable

from aoc2021.util import read_as_list, invert_dict

## Puzzle input data

In [2]:
parse_input = lambda x: tuple(map(str.split, x.rstrip().split('|')))

# Test data.
tdata = list(map(parse_input, [
    'be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe',
    'edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc',
    'fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg',
    'fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb',
    'aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea',
    'fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb',
    'dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe',
    'bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef',
    'egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb',
    'gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce',
]))

# Input data.
data = read_as_list(Path('./day08-input.txt'), func=parse_input)
data[:2]

[(['badc',
   'bd',
   'dbeaf',
   'cfdbge',
   'dfb',
   'cfbdea',
   'efbag',
   'edcfgab',
   'dcafe',
   'degfca'],
  ['eacfd', 'acdfbe', 'cbdegf', 'fcbaedg']),
 (['cd',
   'fdbac',
   'egcfab',
   'gbadcfe',
   'cfgdeb',
   'cbadfe',
   'deca',
   'cdf',
   'dfabg',
   'abefc'],
  ['dcf', 'cfbad', 'gbafced', 'fcd'])]

## Puzzle answers
### Part 1

In [3]:
Input = list[list[str]]


def is_unique(digit: str) -> bool:
    return len(digit) in {2, 3, 4, 7}


def num_unique_output_digits(data: Input) -> int:
    return sum(len(list(filter(is_unique, ods))) for _,ods in data)    


assert all(map(is_unique, ['ab','abc','abcd','abcdefg']))
assert not any(map(is_unique, ['abcde','abcdef']))
assert num_unique_output_digits(tdata) == 26

In [4]:
n = num_unique_output_digits(data)
print(f'Number of times the digits 1, 4, 7, or 8 appear in the output values: {n}')

Number of times the digits 1, 4, 7, or 8 appear in the output values: 387


### Part 2

In [5]:
SEGMENTS = {
    'cf': 1,
    'acf': 7,
    'bcdf': 4,
    'abcdefg': 8,
    'abcefg': 0,
    'acdeg': 2,
    'acdfg': 3,
    'abdfg': 5,
    'abdefg': 6,
    'abcdfg': 9,
}


def decoding_map(digits: list[str]) -> dict[str,str]:
    freqs = Counter(''.join(digits))
    lens = {c: sum(len(d) for d in digits if c in d) for c in 'abcdefg'}
    decoding = {}
    for c in 'abcdefg':
        match freqs[c], lens[c]:
            case 4,_: decoding[c] = 'e'
            case 6,_: decoding[c] = 'b'
            case 9,_: decoding[c] = 'f'
            case 7,38: decoding[c] = 'd'
            case 7,40: decoding[c] = 'g'
            case 8,43: decoding[c] = 'a'
            case 8,38: decoding[c] = 'c'
            case _,_: raise Exception(f'invalid pattern for "{c}": freq={freqs[c]}, len={lens[c]}')
    return decoding


def decoder(digits: list[str]) -> Callable[[str], int]:
    char_decoding = str.maketrans(decoding_map(digits))
    str_decoding = {frozenset(k): v for k, v in SEGMENTS.items()}
    return lambda d: str_decoding[frozenset(d.translate(char_decoding))]


def output_values(data: Input) -> list[int]:
    return [int(''.join(map(str, map(decoder(sds), ods)))) for sds,ods in data]


assert list(map(decoder(SEGMENTS), SEGMENTS)) == list(SEGMENTS.values())
assert output_values(tdata) == [8394,9781,1197,9361,4873,8418,4548,1625,8717,4315]
assert sum(output_values(tdata)) == 61229

In [6]:
n = sum(output_values(data))
print(f'Adding up all of the output values: {n}')

Adding up all of the output values: 986034
